## Introduction

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Overview of the Data

The following will display a preview of the data so we can get a feel for it.

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT *
    FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* `name` — the name of the country.
* `area`— the country's total area (both land and water).
* `area_land` — the country's land area in square kilometers.
* `area_water` — the country's waterarea in square kilometers.
* `population` — the country's population.
* `population_growth`— the country's population growth as a percentage.
* `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
* `death_rate` — the country's death rate, or the number of death per year per 1,000 people.

Let's start by calculating some summary statistics and look for any outlier countries.

# Summary Statistics

In [4]:
%%sql

SELECT MIN(population) AS 'Minimum Population', MAX(population) AS 'Maximum Population', 
MIN(population_growth) AS 'Minimum Population Growth', MAX(population_growth) AS 'Maximum Population Growth'
    FROM facts;

 * sqlite:///factbook.db
Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,7256490011,0.0,4.02


We see a few interesting things in the summary statistics on the previous screen:

There's a country with a population of 0
There's a country with a population of 7256490011 (or more than 7.2 billion people)
Let's use subqueries to zoom in on just these countries without using the specific values.

## Exploring Outliers

In [5]:
%%sql

SELECT * 
    FROM facts
 WHERE population == (SELECT MIN(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [6]:
%%sql

SELECT * 
    FROM facts
 WHERE population == (SELECT Max(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


At first glance, one could guess that the 7.2 billion population was referring to the world. In regards to the populaton of 0, this is the continent of Anarctica. The CIA factbook entry on [Antarctica](https://www.cia.gov/the-world-factbook/countries/antarctica/#people-and-society) appears to confirm this fact.


![img](...) 

## Maximum and Minimum Population

In [7]:
%%sql

SELECT MIN(population) AS 'Minimum Population', 
MAX(population) AS 'Maximum Population', 
MIN(population_growth) AS 'Minimum Population Growth', 
MAX(population_growth) AS 'Maximum Population Growth'
    FROM facts
    WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,1367485388,0.0,4.02


## Average Population and Average Area

In [8]:
%%sql
SELECT AVG(population) as Avg_pop, AVG(area) as Avg_area
    FROM facts
    WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


Avg_pop,Avg_area
32242666.56846473,555093.546184739


In [9]:
%%sql

SELECT *
    FROM facts
    WHERE population > (SELECT AVG(population) FROM facts WHERE name <> 'World') AND 
    area < (SELECT AVG(area) FROM facts WHERE name <> 'World');


 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


## Country with Highest Population

In [10]:
%%sql

SELECT name, MAX(population) as Highest_Population
    FROM facts
    WHERE name <> 'World';  

 * sqlite:///factbook.db
Done.


name,Highest_Population
China,1367485388


## Country with Highest Population Growth

In [11]:
%%sql

SELECT name, MAX(population_growth) as Highest_Population_Growth
    FROM facts
    WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


name,Highest_Population_Growth
South Sudan,4.02


## Water to land ratio

In [12]:
%%sql

SELECT name, ROUND(CAST(area_water as Float) / CAST(area_land as Float), 2) as water_land_ratio
    FROM facts
    WHERE name <> 'World'
    ORDER BY water_land_ratio DESC
    LIMIT 10;

 * sqlite:///factbook.db
Done.


name,water_land_ratio
British Indian Ocean Territory,905.67
Virgin Islands,4.52
Puerto Rico,0.55
"Bahamas, The",0.39
Guinea-Bissau,0.28
Malawi,0.26
Netherlands,0.23
Uganda,0.22
Eritrea,0.16
Liberia,0.16


## Land to water ratio

In [13]:
%%sql

SELECT name, ROUND(CAST(area_land as Float) / CAST(area_water as Float), 2) as 
land_to_water_ratio
    FROM facts
    WHERE name <> 'World'
    ORDER BY land_to_water_ratio DESC
    LIMIT 10;

 * sqlite:///factbook.db
Done.


name,land_to_water_ratio
Bosnia and Herzegovina,5118.7
Niger,4222.33
Morocco,1785.2
Guinea,1755.12
Costa Rica,1276.5
Djibouti,1159.0
"Korea, North",926.22
Cyprus,924.1
Namibia,821.65
Burkina Faso,684.5


## Countries with a death rate greater than their birth rate

The countries below are listed in descending order by death rate. 

In [14]:
%%sql

SELECT name as country_name, birth_rate, death_rate
    FROM facts
    WHERE death_rate > birth_rate
    ORDER BY death_rate DESC;

 * sqlite:///factbook.db
Done.


country_name,birth_rate,death_rate
Ukraine,10.72,14.46
Bulgaria,8.92,14.44
Latvia,10.0,14.31
Lithuania,10.1,14.27
Russia,11.6,13.69
Serbia,9.08,13.66
Belarus,10.7,13.36
Hungary,9.16,12.73
Moldova,12.0,12.59
Estonia,10.51,12.4


## Exploring Average Population and Area

Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.

We should take care of discarding the row for the whole planet.

In [15]:
%%sql
SELECT AVG(population) AS avg_population, AVG(area) AS avg_area
  FROM facts
 WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


avg_population,avg_area
32242666.56846473,555093.546184739


## Finding Densely Populated Countries

To finish, we'll build on the query above to find countries that are densely populated. We'll identify countries that have:

Above average values for population.
Below average values for area.

In [16]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name <> 'World'
                    )
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name <> 'World'
);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


## Population Density Ratio

Top 10 countries with the highest population density ratio listed in descending order.

In [17]:
%%sql

SELECT name, ROUND(CAST(population as Float) / CAST(area as Float), 2) as population_density_ratio
    FROM facts
    WHERE name <> 'World'
    ORDER BY population_density_ratio DESC
    LIMIT 10;

 * sqlite:///factbook.db
Done.


name,population_density_ratio
Macau,21168.96
Monaco,15267.5
Singapore,8141.28
Hong Kong,6445.04
Gaza Strip,5191.82
Gibraltar,4876.33
Bahrain,1771.86
Maldives,1319.64
Malta,1310.02
Bermuda,1299.93
